## 0. Import Modules and Data

In [1]:
import pandas as pd
import numpy as np
from pandas_datareader import data
import pandas_datareader.data as web
import datetime as dt

In [2]:
import Portfolio as port # This is from the Portfolio.py file with the different classes defined

## 1. Set-up Portfolio Strategy

In [3]:
# Define start date and end date

start_date = '2015-01-01'

end_date = '2020-01-05'

In [4]:
# Define Shares

# Provide the name of the ticker and type (Equity or Bond)

voo = port.Share('VOO', 'Equity', start_date,end_date)
bnd = port.Share('BND', 'Bond', start_date, end_date)

In [5]:
voo.get_value('2019-01-01')

229.99

In [6]:
voo.get_value('2019-10-31')

278.55

In [7]:
# Need a way to group the different shares together

shares_list = [voo, bnd]

shares_dict = {}

for share in shares_list:
    shares_dict[share] = share.type


In [8]:
shares_dict

{<Portfolio.Share at 0x7fcfa8bbe0f0>: 'Equity',
 <Portfolio.Share at 0x7fcfd46b56d8>: 'Bond'}

In [9]:
# Define Strategy

# Provide the equity distribution, the bond distriubtion, cash distribution, and the threshold
strat = port.Strategy(50,50,0,5)

In [10]:
# Run portfolio with set start and end dates

portfolio = port.Portfolio(shares_dict)

portfolio.initial_buy(500, strat, start_date)

In [11]:
portfolio.get_asset_values(start_date)

In [12]:
portfolio.asset_split

{'Equities': 50.0, 'Bonds': 50.0, 'Cash': 0.0}

In [13]:
# Run the portfolio over a series of months

time_period = pd.date_range(pd.to_datetime(start_date),pd.to_datetime(end_date))

for day in time_period:
    print(day)
    portfolio.reinvest_divs(day)
    portfolio.get_asset_values(day)
    print(portfolio.asset_split)
    if portfolio.asset_split['Equities'] > strat.equity_distribution+strat.threshold:
        sell_amt = (portfolio.asset_values['Equities']+portfolio.asset_values['Bonds'])*((portfolio.asset_split['Equities']-strat.equity_distribution)/100)
        sell_amt_per = sell_amt/len(portfolio.equities)
        for share in portfolio.equities: # sell equities and buy more bonds
            portfolio.sell(share, sell_amt_per, day)
        for share in portfolio.bonds:
            portfolio.buy(share, sell_amt_per, day)
               
    if portfolio.asset_split['Bonds'] > strat.bond_distribution+strat.threshold:
        sell_amt = (portfolio.asset_values['Equities']+portfolio.asset_values['Bonds'])*(portfolio.asset_split['Bonds']-strat.bond_distribution)
        sell_amt_per = sell_amt/len(portfolio.bonds)
        for share in portfolio.bonds: # sell bonds and buy more equities
            portfolio.sell(share, sell_amt_per, day)
        for share in portfolio.bonds:
            portfolio.buy(share, sell_amt_per, day)
        
    

2015-01-01 00:00:00
{'Equities': 50.0, 'Bonds': 50.0, 'Cash': 0.0}
2015-01-02 00:00:00
{'Equities': 50.0, 'Bonds': 50.0, 'Cash': 0.0}
2015-01-03 00:00:00
{'Equities': 49.48439897186935, 'Bonds': 50.51560102813064, 'Cash': 0.0}
2015-01-04 00:00:00
{'Equities': 49.48439897186935, 'Bonds': 50.51560102813064, 'Cash': 0.0}
2015-01-05 00:00:00
{'Equities': 49.48439897186935, 'Bonds': 50.51560102813064, 'Cash': 0.0}
2015-01-06 00:00:00
{'Equities': 49.16513484755205, 'Bonds': 50.83486515244794, 'Cash': 0.0}
2015-01-07 00:00:00
{'Equities': 49.460491131767945, 'Bonds': 50.53950886823204, 'Cash': 0.0}
2015-01-08 00:00:00
{'Equities': 49.934970195913024, 'Bonds': 50.06502980408698, 'Cash': 0.0}
2015-01-09 00:00:00
{'Equities': 49.68145734582606, 'Bonds': 50.31854265417394, 'Cash': 0.0}
2015-01-10 00:00:00
{'Equities': 49.462145782702805, 'Bonds': 50.537854217297195, 'Cash': 0.0}
2015-01-11 00:00:00
{'Equities': 49.462145782702805, 'Bonds': 50.537854217297195, 'Cash': 0.0}
2015-01-12 00:00:00
{'E

2015-05-07 00:00:00
{'Equities': 50.73575178410514, 'Bonds': 49.59570561100386, 'Cash': -0.3314573951089926}
2015-05-08 00:00:00
{'Equities': 50.994044911577355, 'Bonds': 49.334788529195876, 'Cash': -0.32883344077323085}
2015-05-09 00:00:00
{'Equities': 51.00554247390382, 'Bonds': 49.324949835765224, 'Cash': -0.3304923096690412}
2015-05-10 00:00:00
{'Equities': 51.00554247390382, 'Bonds': 49.324949835765224, 'Cash': -0.3304923096690412}
2015-05-11 00:00:00
{'Equities': 51.00554247390382, 'Bonds': 49.324949835765224, 'Cash': -0.3304923096690412}
2015-05-12 00:00:00
{'Equities': 50.96364178160802, 'Bonds': 49.36753987298609, 'Cash': -0.33118165459410487}
2015-05-13 00:00:00
{'Equities': 50.983725278265325, 'Bonds': 49.34748363133469, 'Cash': -0.331208909600017}
2015-05-14 00:00:00
{'Equities': 51.194126081036764, 'Bonds': 49.13501329802523, 'Cash': -0.32913937906199225}
2015-05-15 00:00:00
{'Equities': 51.11877258057484, 'Bonds': 49.20954478332725, 'Cash': -0.3283173639021024}
2015-05-16

2015-09-22 00:00:00
{'Equities': 49.431073882909324, 'Bonds': 51.43656254433473, 'Cash': -0.8676364272440568}
2015-09-23 00:00:00
{'Equities': 49.38603259548714, 'Bonds': 51.48237662771481, 'Cash': -0.8684092232019337}
2015-09-24 00:00:00
{'Equities': 49.29908710592636, 'Bonds': 51.570488782108036, 'Cash': -0.869575888034398}
2015-09-25 00:00:00
{'Equities': 49.328874241674484, 'Bonds': 51.54181775531025, 'Cash': -0.8706919969847265}
2015-09-26 00:00:00
{'Equities': 48.62629886756045, 'Bonds': 52.254157976656316, 'Cash': -0.8804568442167621}
2015-09-27 00:00:00
{'Equities': 48.62629886756045, 'Bonds': 52.254157976656316, 'Cash': -0.8804568442167621}
2015-09-28 00:00:00
{'Equities': 48.62629886756045, 'Bonds': 52.254157976656316, 'Cash': -0.8804568442167621}
2015-09-29 00:00:00
{'Equities': 48.62069231750738, 'Bonds': 52.259203428913445, 'Cash': -0.8798957464208245}
2015-09-30 00:00:00
{'Equities': 49.046190772324536, 'Bonds': 51.824684873294245, 'Cash': -0.8708756456187801}
2015-10-01 

{'Equities': 49.03954397568405, 'Bonds': 52.253189836502834, 'Cash': -1.292733812186878}
2016-02-14 00:00:00
{'Equities': 49.03954397568405, 'Bonds': 52.253189836502834, 'Cash': -1.292733812186878}
2016-02-15 00:00:00
{'Equities': 49.03954397568405, 'Bonds': 52.253189836502834, 'Cash': -1.292733812186878}
2016-02-16 00:00:00
{'Equities': 49.03954397568405, 'Bonds': 52.253189836502834, 'Cash': -1.292733812186878}
2016-02-17 00:00:00
{'Equities': 49.4518522235413, 'Bonds': 51.83074363811456, 'Cash': -1.2825958616558608}
2016-02-18 00:00:00
{'Equities': 49.258115026724994, 'Bonds': 52.0248968118381, 'Cash': -1.283011838563077}
2016-02-19 00:00:00
{'Equities': 49.28677958302071, 'Bonds': 51.99727052021761, 'Cash': -1.2840501032383178}
2016-02-20 00:00:00
{'Equities': 49.635764578689056, 'Bonds': 51.639130577259685, 'Cash': -1.2748951559487516}
2016-02-21 00:00:00
{'Equities': 49.635764578689056, 'Bonds': 51.639130577259685, 'Cash': -1.2748951559487516}
2016-02-22 00:00:00
{'Equities': 49.6

2016-07-19 00:00:00
{'Equities': 51.97933949591598, 'Bonds': 49.75257350549434, 'Cash': -1.7319130014103217}
2016-07-20 00:00:00
{'Equities': 52.10744700727376, 'Bonds': 49.62176042464088, 'Cash': -1.7292074319146513}
2016-07-21 00:00:00
{'Equities': 51.98255966207156, 'Bonds': 49.74902415783395, 'Cash': -1.7315838199055147}
2016-07-22 00:00:00
{'Equities': 52.094950816107534, 'Bonds': 49.63236751136115, 'Cash': -1.7273183274686754}
2016-07-23 00:00:00
{'Equities': 52.050133825362806, 'Bonds': 49.67986445174837, 'Cash': -1.7299982771111804}
2016-07-24 00:00:00
{'Equities': 52.050133825362806, 'Bonds': 49.67986445174837, 'Cash': -1.7299982771111804}
2016-07-25 00:00:00
{'Equities': 52.050133825362806, 'Bonds': 49.67986445174837, 'Cash': -1.7299982771111804}
2016-07-26 00:00:00
{'Equities': 52.05195066754911, 'Bonds': 49.67819500380146, 'Cash': -1.7301456713505798}
2016-07-27 00:00:00
{'Equities': 51.958934695665825, 'Bonds': 49.76924910682053, 'Cash': -1.7281838024863494}
2016-07-28 00:

...

{'Equities': 48.714173238255725, 'Bonds': 56.13790810751148, 'Cash': -4.852081345767195}
2018-12-26 00:00:00
{'Equities': 48.714173238255725, 'Bonds': 56.13790810751148, 'Cash': -4.852081345767195}
2018-12-27 00:00:00
{'Equities': 48.88627373647444, 'Bonds': 55.938111073354136, 'Cash': -4.824384809828581}
2018-12-28 00:00:00
{'Equities': 48.78864998857663, 'Bonds': 56.032231214228304, 'Cash': -4.820881202804939}
2018-12-29 00:00:00
{'Equities': 48.94343644767256, 'Bonds': 55.849598758209375, 'Cash': -4.793035205881948}
2018-12-30 00:00:00
{'Equities': 48.94343644767256, 'Bonds': 55.849598758209375, 'Cash': -4.793035205881948}
2018-12-31 00:00:00
{'Equities': 48.94343644767256, 'Bonds': 55.849598758209375, 'Cash': -4.793035205881948}
2019-01-01 00:00:00
{'Equities': 48.90784821774429, 'Bonds': 55.87795332303342, 'Cash': -4.785801540777686}
2019-01-02 00:00:00
{'Equities': 48.90784821774429, 'Bonds': 55.87795332303342, 'Cash': -4.785801540777686}
2019-01-03 00:00:00
{'Equities': 48.19788

{'Equities': 52.50641147328212, 'Bonds': 52.21761734026328, 'Cash': -4.724028813545396}
2019-04-25 00:00:00
{'Equities': 52.503900905810184, 'Bonds': 52.22166286974641, 'Cash': -4.725563775556593}
2019-04-26 00:00:00
{'Equities': 52.55698929749655, 'Bonds': 52.1517633326281, 'Cash': -4.70875263012465}
2019-04-27 00:00:00
{'Equities': 52.64600389035044, 'Bonds': 52.06478074347266, 'Cash': -4.7107846338230965}
2019-04-28 00:00:00
{'Equities': 52.64600389035044, 'Bonds': 52.06478074347266, 'Cash': -4.7107846338230965}
2019-04-29 00:00:00
{'Equities': 52.64600389035044, 'Bonds': 52.06478074347266, 'Cash': -4.7107846338230965}
2019-04-30 00:00:00
{'Equities': 52.62104960089615, 'Bonds': 52.08453757965453, 'Cash': -4.70558718055069}
2019-05-01 00:00:00
{'Equities': 52.5028728144864, 'Bonds': 52.259421344299504, 'Cash': -4.762294158785914}
2019-05-02 00:00:00
{'Equities': 52.51473617926218, 'Bonds': 52.2580714034932, 'Cash': -4.772807582755397}
2019-05-03 00:00:00
{'Equities': 52.699330776791

{'Equities': 50.727937379223945, 'Bonds': 54.29542865060356, 'Cash': -5.023366029827498}
2019-08-16 00:00:00
{'Equities': 51.12066808293172, 'Bonds': 53.86848510965768, 'Cash': -4.989153192589406}
2019-08-17 00:00:00
{'Equities': 51.51365889944848, 'Bonds': 53.452802310999395, 'Cash': -4.966461210447874}
2019-08-18 00:00:00
{'Equities': 51.51365889944848, 'Bonds': 53.452802310999395, 'Cash': -4.966461210447874}
2019-08-19 00:00:00
{'Equities': 51.51365889944848, 'Bonds': 53.452802310999395, 'Cash': -4.966461210447874}
2019-08-20 00:00:00
{'Equities': 51.247823032991334, 'Bonds': 53.73158585536546, 'Cash': -4.979408888356805}
2019-08-21 00:00:00
{'Equities': 51.46255353926305, 'Bonds': 53.497502607615374, 'Cash': -4.960056146878418}
2019-08-22 00:00:00
{'Equities': 51.50528635493698, 'Bonds': 53.460737589952814, 'Cash': -4.966023944889786}
2019-08-23 00:00:00
{'Equities': 50.75487534127492, 'Bonds': 54.2676995606315, 'Cash': -5.022574901906425}
2019-08-24 00:00:00
{'Equities': 51.046659

{'Equities': 53.7706329802546, 'Bonds': 51.344668793870994, 'Cash': -5.11530177412559}
2019-12-07 00:00:00
{'Equities': 53.68829064129095, 'Bonds': 51.43476712395102, 'Cash': -5.123057765241957}
2019-12-08 00:00:00
{'Equities': 53.68829064129095, 'Bonds': 51.43476712395102, 'Cash': -5.123057765241957}
2019-12-09 00:00:00
{'Equities': 53.68829064129095, 'Bonds': 51.43476712395102, 'Cash': -5.123057765241957}
2019-12-10 00:00:00
{'Equities': 53.684729592335515, 'Bonds': 51.442790250745986, 'Cash': -5.127519843081482}
2019-12-11 00:00:00
{'Equities': 53.657423039807796, 'Bonds': 51.454179092663345, 'Cash': -5.111602132471141}
2019-12-12 00:00:00
{'Equities': 54.00647878839153, 'Bonds': 51.0935195605057, 'Cash': -5.0999983488972305}
2019-12-13 00:00:00
{'Equities': 53.8910364871971, 'Bonds': 51.1961394799306, 'Cash': -5.087175967127699}
2019-12-14 00:00:00
{'Equities': 54.14144537076987, 'Bonds': 50.93349708348972, 'Cash': -5.074942454259582}
2019-12-15 00:00:00
{'Equities': 54.14144537076

OutOfBoundsDatetime: Out of bounds nanosecond timestamp 9223372800000000000

In [14]:
portfolio.cash_bal

-33.958000000000176

In [15]:
portfolio.log

{0: ['Buy',
  250.0,
  188.4,
  1.326963906581741,
  '2015-01-01'],
 1: ['Buy',
  250.0,
  82.65,
  3.024803387779794,
  '2015-01-01'],
 2: ['Buy',
  0.17,
  84.08,
  0.002021883920076118,
  Timestamp('2015-02-02 00:00:00', freq='D')],
 3: ['Buy',
  0.162,
  82.65,
  0.0019600725952813067,
  Timestamp('2015-03-02 00:00:00', freq='D')],
 4: ['Buy',
  0.984,
  192.52,
  0.005111157282360274,
  Timestamp('2015-03-23 00:00:00', freq='D')],
 5: ['Buy',
  0.191,
  83.51,
  0.0022871512393725303,
  Timestamp('2015-04-01 00:00:00', freq='D')],
 6: ['Buy',
  0.159,
  82.5,
  0.0019272727272727273,
  Timestamp('2015-05-01 00:00:00', freq='D')],
 7: ['Buy',
  0.166,
  81.9,
  0.002026862026862027,
  Timestamp('2015-06-01 00:00:00', freq='D')],
 8: ['Buy',
  0.902,
  194.27,
  0.0046430225974159675,
  Timestamp('2015-06-22 00:00:00', freq='D')],
 9: ['Buy',
  0.163,
  80.9,
  0.002014833127317676,
  Timestamp('2015-07-01 00:00:00', freq='D')],
 10: ['Buy',
  0.169,
  81.83,
  0.0020652572406207994

In [ ]:
sell_amt

In [ ]:
portfolio.shares

In [ ]:
portfolio.log

In [ ]:
portfolio.asset_split['Bonds']-strat.bond_distribution

In [ ]:
portfolio.asset_split['Bonds']

In [ ]:
246.38667/.502971399

In [ ]:
(246.38667+243.47552)*.002971399

In [ ]:
489.86218/2

In [ ]:
246.38667*.002971399

In [ ]:
output

In [ ]:
portfolio.log

In [ ]:
time_period

In [ ]:
start_date

In [ ]:
# Get the value a few days later

portfolio.get_value('2018-05-01')

In [ ]:
portfolio.get_asset_values('2018-05-01')

In [ ]:
portfolio.asset_values

In [ ]:
for share in portfolio.shares:
    print(share.div.index)

In [ ]:
portfolio.reinvest_divs('2018-05-01')

In [ ]:
# import sys
# !{sys.executable} -m pip install pandas_datareader

In [ ]:
## This is useful:
#https://jakevdp.github.io/PythonDataScienceHandbook/03.11-working-with-time-series.html

In [ ]:
bnd = web.DataReader('BND', 'yahoo', start='2015-10-30', end='2020-10-30')

In [ ]:
sp = web.DataReader('VOO', 'yahoo', start='2015-10-30', end='2020-10-30')

In [ ]:
sp.head(3)

In [ ]:
bnd.head(3)

In [ ]:
bnd.loc['2017-07-07']['High']

In [ ]:
# Import dividend records
bnd_div = web.DataReader('BND', 'yahoo-dividends', start='2015-10-30', end='2020-10-30')
sp_div = web.DataReader('VOO', 'yahoo-dividends', start='2015-10-30', end='2020-10-30')

In [ ]:
bnd_div.head(3)

In [ ]:
sp_div.head(3)

In [ ]:
sp_div.value.loc['2020-09-29']

In [ ]:
sp_div[sp_div.index < pd.to_datetime('2016-01-01')]

In [ ]:
sp_div.index

In [ ]:
str('sp')+str('_div')

In [ ]:
Assets = {}
Assets['Bonds'] = bnd
Assets['Equities'] = sp

In [ ]:
Dividends

In [ ]:
# Maybe it is worth merging the dividend and share price files together for easier reference?
# Or build a dictionary to allow easy mapping?

## 1. Calculate Value Given a Start and End Date

In [ ]:
start_date = '2015-11-20'
end_date = 

In [ ]:
def calc_value(share, divs, start, end, initial_value):
    # Take the average of the day's high and low for starting share price
    num_shares = initial_value/np.mean([share.loc[start]['Low'],share.loc[start]['High']])
    divs = divs[(divs.index <= pd.to_datetime(end)) & (divs.index >= pd.to_datetime(start))]
    for date in divs.index:
        price = np.mean([share.High.loc[date], share.Low.loc[date]])
        div_value = divs.value.loc[date]
        num_shares += div_value/price
    output = {}
    output['Value'] = num_shares*np.mean([share.loc[end]['Low'],share.loc[end]['High']])
    output['ROI'] = output['Value']-initial_value
    return(output)
    
    # Will need to factor in other actions later, but this will work for now
    
    


In [ ]:
calc_value(sp,sp_div, '2015-11-20', '2017-11-20',100)

In [ ]:
calc_value(bnd,bnd_div, '2015-11-20', '2017-11-20',100)

In [ ]:
sp.Close.plot()

In [ ]:
bnd.Close.plot()

In [ ]:
# Identify the ROI with different asset distributions

In [ ]:
class Share:

    def __init__(self, name, asset_type):
        """Initialize attributes."""
        self.name = name
        self.type = asset_type
        #self.amount = amount
        
    def get_value(self, date):
        df = web.DataReader(self.name, 'yahoo', start=date, end=date)
        return round(df.loc[date,'Close'],2)
    

In [ ]:
bnd = Share('BND','Bond')

In [ ]:
sp = Share('VOO','Equity')

In [ ]:
class Strategy:
    
    def __init__(self,equity_distribution, bond_distribution, cash_distribution, threshold):
        self.equity_distribution = equity_distribution
        self.bond_distribution = bond_distribution
        self.cash_distribution = 100-(equity_distribution+bond_distribution)
        self.threshold = threshold
    

In [ ]:
class Portfolio:
    
    def __init__(self):
        self.shares = {}
        self.log = {}
        self.cash_bal = 0
        self.asset_split = {'Equities': None,'Bonds': None,'Cash': None}
        self.asset_values = {'Equities': None,'Bonds': None,'Cash': None}
        
    def buy(self, share, amount, date):
        purchase_price = share.get_value(date)
        shares = amount/purchase_price
        # Record the transaction
        self.log[len(self.log)] = ['Buy',share,amount,purchase_price,shares,date]
        # Add the number of shares
        if share in self.shares:
            self.shares[share] += shares
        elif share not in self.shares:
            self.shares[share] = shares
            
    def sell(self, share, amount, date):
        sell_price = share.get_value(date)
        shares = amount/sell_price
        # Record the transaction
        self.log[len(self.log)] = ['Sell',share,amount,sell_price,shares,date]
        # Add the number of shares
        if share in self.shares:
            self.shares[share] -= shares
        elif share not in self.shares:
            self.shares[share] = shares
        cash_bal += amount
        
            
    def get_value(self,date):
        value = self.cash_bal
        # Iterate through the 'shares' attribute and get the current value for each 'Share' object
        for share in self.shares:
            value += share.get_value(date)*self.shares[share]
        return(value)
    
    def get_asset_values(self,date):
        bond_val = 0
        eq_val = 0
        for share in self.shares:
            if share.type == 'Bond':
                bond_val += share.get_value(date)*self.shares[share]
            elif share.type == 'Equity':
                eq_val += share.get_value(date)*self.shares[share]
        total = bond_val+eq_val+self.cash_bal
        self.asset_values = {'Equities': eq_val,'Bonds': bond_val,'Cash': self.cash_bal}
        self.asset_split = {'Equities': (eq_val/total)*100,'Bonds': (bond_val/total)*100,'Cash': (self.cash_bal/total)*100}
            
        
    

In [ ]:
portfolio = Portfolio()

In [ ]:
portfolio.get_value)

In [ ]:
# Calculate portfolio value

start_date = '2015-11-20'
end_date = '2020-11-20'

portfolio.buy(sp,750,start_date)
portfolio.buy(bnd,250,start_date)

portfolio.get_value(end_date)

In [ ]:
bond = web.DataReader('BND', 'yahoo', start='2015-10-30', end='2020-10-30')

In [ ]:
dates = list(bond.index)

In [ ]:
dates = [x.strftime('%Y-%m-%d') for x in dates]

In [ ]:
# Deploy balancing strategy

strategy = Strategy(75,25,0,1)

# Initial Purchase (manually allocated proportions)
portfolio = Portfolio() 
portfolio.buy(sp,750,start_date)
portfolio.buy(bnd,250,start_date)

for date in dates:
    portfolio.get_asset_values(date)
    if portfolio.asset_split['Equities'] > strategy.equity_distribution+strategy.threshold:
        amount_to_sell = (portfolio.asset_split['Equities']-strategy.equity_distribution)*portfolio.asset_values['Equities']
        portfolio.sell(sp,amount_to_sell,date)
        portfolio.buy(bnd,amount_to_sell,date)
        

    

In [ ]:
portfolio.get_asset_values('2019-10-29')

In [ ]:
portfolio.asset_split['Equities']